In [ ]:
import numpy as np
import tensorflow as tf
from keras import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten

## Parameters

In [ ]:
# General

n_agents = 2

# Model

time_steps = 3
grid_size_w = 20
grid_size_h = 20
n_actions = 5

n_channels = time_steps + time_steps + 1

# Q-Learning

n_episodes = 10
n_steps = 100
epsilon = 0.75
epsilon_decay = 0.999
gamma = 0.9

## Model

In [ ]:
model = Sequential()
model.add(Conv2D(16, (4, 4), activation="relu", input_shape=(grid_size_w, grid_size_h, n_channels)))
model.add(Conv2D(32, (2, 2), activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(n_actions, activation="linear"))
model.compile(optimizer="adam", loss="mse")
model.summary()

## Q-Learning

In [ ]:
X = np.random.rand(n_agents, grid_size_w, grid_size_h, n_channels)

# todo: initialize environment

for i in range(n_episodes):
    if i % 10 == 0:
        print("Episode {} of {}".format(i + 1, n_episodes))
    epsilon *= epsilon_decay
    r_sum = 0
    step = 0
    while step < n_steps:
        if np.random.random() < epsilon:
            actions = np.random.randint(0, n_actions)
        else:
            # todo: get current known state as X
            actions = np.argmax(model.predict(X), axis=1)
        # todo: do action, get r and new state as X_new
        
        # after all agents move
        rewards = np.floor(np.random.rand(n_agents) * 10)
        X_new = np.random.rand(n_agents, grid_size_w, grid_size_h, n_channels)
        
        target = model.predict(X)
        target[:, actions] = rewards + gamma * np.max(model.predict(X_new), axis=1)
        
        model.fit(X, target, epochs=1, verbose=0)
        step += 1